In [1]:
import numpy as np
import wave
#import matplotlib.pyplot as plt

In [ ]:
# function definitions

def make_wave(freq, sample_rate, harmonics, note_length, decay = lambda x: 1):
  """
  Creates and returns a numpy array representing an audio wave of given specifications
  Arguments:
    freq: note frequency in Hz
    sample_rate: sample rate in kHz
    harmonics: array denoting coefficients to apply harmonics to. Ex. [.6, .2] will apply .6*1st harmonic wave + .2*2nd harmonic wave
    note_length: note length in seconds
    decay: function which returns a scaling coefficient on a single audio sample
  """
  N = len(harmonics)  # N = num harmonics
  x = np.linspace(0, note_length, int(sample_rate*note_length))

  audio = np.sum([harmonics[n-1] * np.sin(n * 2 * np.pi *freq * x) for n in range(1, N+1)], axis=0) # generate wave
  audio = np.array([decay(i)*val for i, val in enumerate(audio)]) # apply decay
  #plt.plot(x, audio)
  #plt.show()

  return audio

def envelope(x, d, s, r, sample_rate):
  """
  Applies a DSR (delay-sustain-release) envelope to a given sample index x
  Returns a scalar to be used as a scaling coefficient on a single audio sample
  Arguments:
    x: index of sample
    d, s, r: delay, sustain, and release in seconds (respectively)
  """
  if x < sample_rate*d: #decay
    return np.exp(-x/(sample_rate**.6)) + .25
  elif x < sample_rate*s: # sustain
    return .25 # sustain vol
  elif x < sample_rate*r: # release
    return .25-.25/(sample_rate*(r-s))*(x-sample_rate*s)
  else:
    return 0
  
def add_padding(audio, pre_gap, total_len, sample_rate=44100):
  """
  Adds 0's before and after a given audio signal array to reach a specified length of time (in seconds). 
  Returns numpy array representing the audio file
  Arguments:
    audio: numpy array representing audio signal
    pre_gap: amount of offset before beginning of audio in seconds
    total_len = desired total audio signal runtime in seconds
  """
  # Satisfies: (pre_samples + len(audio) + post_samples) == int(total_len * sample_rate)
  pre_samples = int(pre_gap*sample_rate)
  post_samples = int(total_len*sample_rate) - len(audio) - pre_samples
  return np.concatenate( (np.zeros(pre_samples), audio, np.zeros(post_samples)) )

def audio_to_wav(audio, fname="sound", sample_rate=44100):
  """"
  Saves array representing audio as wav file in mono
  Arguments:
    audio: array representing audio signal
    fname: name of file to save as
    sample_rate: sample rate in kHz
  """
  with wave.open(fname + ".wav", "w") as obj:
    obj.setnchannels(1) # mono
    obj.setsampwidth(2) # 2 bytes per sample
    obj.setframerate(sample_rate)

    post_audio = (audio * (2 ** 15 - 1)).astype("<h") # convert to (little-endian) 16 bit ints
    obj.writeframes(post_audio.tobytes())

In [ ]:
G4_freq = 392.00
E5_freq = 659.25	
C5_freq = 523.25
sample_rate = 44100 

total_len = 2 # in secs
gap = .4 # gap between notes in seconds

harmonics = [.65, .10, .07, .05, .03]

# make notes
g4 = make_wave(G4_freq, sample_rate, harmonics, 2, decay = lambda x: envelope(x, .25, .35, 1, sample_rate*2))
e5 = make_wave(E5_freq, sample_rate, harmonics, 1, decay = lambda x: envelope(x, .25, .55, 1, sample_rate*1))
c5 = make_wave(C5_freq, sample_rate, harmonics, 1.2, decay = lambda x: envelope(x, .25, .55, 1, sample_rate*1.2))

# offset c5 and e5
c5_padded = add_padding(c5, pre_gap = 2*gap, total_len = total_len)
e5_padded = add_padding(e5, pre_gap = gap, total_len = total_len)

# combine and export
nbc_audio = g4 + c5_padded + e5_padded
audio_to_wav(nbc_audio, fname = "nbc")